# 🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo

Objetivo: implementar un pipeline near-real-time que ingiere eventos en Kafka, los valida y transforma, escribe salidas particionadas en Parquet y expone métricas básicas de procesamiento.

- Duración: 120–150 min
- Dificultad: Media/Alta
- Prerrequisitos: Notebooks Mid 02 (Kafka), 05 (DataOps)

## 0) Requisitos y ejecución

- Necesitas un clúster Kafka en local (Docker Compose) o remoto.
- Dependencias opcionales: `kafka-python` o `confluent-kafka`. No están activas por defecto en `requirements.txt`.
- Este notebook incluye un modo de simulación sin Kafka para que puedas practicar la lógica de validación/transformación/sink.
- Variables de entorno: `KAFKA_BOOTSTRAP_SERVERS`, `KAFKA_TOPIC`, `OUT_DIR` (por defecto `datasets/processed/pi2/`).

Ejemplo Docker Compose (para referencia):
```yaml
version: '3.8'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:7.4.0
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
  kafka:
    image: confluentinc/cp-kafka:7.4.0
    depends_on: [zookeeper]
    ports: ['9092:9092']
    environment:
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:2181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:9092,PLAINTEXT_HOST://localhost:9092
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
```

## 1) Esquema de eventos y validación

In [ ]:
from cerberus import Validator
event_schema = {
  'event_id': {'type':'string', 'required': True},
  'ts': {'type':'string', 'required': True},
  'usuario_id': {'type':'integer', 'required': True},
  'accion': {'type':'string', 'allowed':['click','view','purchase']},
  'monto': {'type':'float', 'nullable': True, 'min': 0}
}
validator = Validator(event_schema, allow_unknown=True)
def is_valid_event(evt):
    return validator.validate(evt)

# Ejemplo
is_valid_event({'event_id':'e1','ts':'2025-10-30T12:00:00Z','usuario_id':1,'accion':'click','monto':None})

## 2) Productor (Kafka) y modo simulación

In [ ]:
import os, json, time, random
from datetime import datetime, timezone
KAFKA_BOOTSTRAP = os.getenv('KAFKA_BOOTSTRAP_SERVERS','localhost:9092')
KAFKA_TOPIC = os.getenv('KAFKA_TOPIC','pi2_events')

def gen_event(i: int):
    return {
        'event_id': f'e{i}',
        'ts': datetime.now(timezone.utc).isoformat(),
        'usuario_id': random.randint(1,1000),
        'accion': random.choice(['click','view','purchase']),
        'monto': round(random.uniform(1,500),2) if random.random()>0.8 else None
    }

def produce_simulation(n=50):
    return [gen_event(i) for i in range(n)]

# Productor Kafka (opcional)
def produce_kafka(n=50):
    try:
        from kafka import KafkaProducer
        producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP, value_serializer=lambda v: json.dumps(v).encode('utf-8'))
        for i in range(n):
            evt = gen_event(i)
            producer.send(KAFKA_TOPIC, evt)
        producer.flush()
        return n
    except Exception as e:
        print('Kafka no disponible, usa modo simulación:', e)
        return None

simulated = produce_simulation(100)
len(simulated)

## 3) Consumidor/Procesador: validación, enriquecimiento e idempotencia

In [ ]:
from loguru import logger
import sqlite3
from typing import Iterable, Dict, Any

OUT_DIR = os.getenv('OUT_DIR','datasets/processed/pi2/')
os.makedirs(OUT_DIR, exist_ok=True)
CKPT_DB = os.path.join(OUT_DIR, 'checkpoint.sqlite')

def ensure_ckpt():
    conn = sqlite3.connect(CKPT_DB)
    cur = conn.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS seen (event_id TEXT PRIMARY KEY)')
    conn.commit(); conn.close()

def is_seen(event_id: str) -> bool:
    conn = sqlite3.connect(CKPT_DB)
    cur = conn.cursor()
    cur.execute('SELECT 1 FROM seen WHERE event_id=?', (event_id,))
    row = cur.fetchone()
    conn.close()
    return row is not None

def mark_seen(event_id: str):
    conn = sqlite3.connect(CKPT_DB)
    cur = conn.cursor()
    cur.execute('INSERT OR IGNORE INTO seen(event_id) VALUES (?)', (event_id,))
    conn.commit(); conn.close()

def enrich(evt: Dict[str,Any]) -> Dict[str,Any]:
    evt = dict(evt)
    evt['processing_ts'] = datetime.now(timezone.utc).isoformat()
    evt['monto'] = float(evt['monto']) if evt.get('monto') is not None else 0.0
    return evt

def process_events(events: Iterable[Dict[str,Any]]):
    ensure_ckpt()
    good, bad = [], []
    for evt in events:
        if not is_valid_event(evt):
            bad.append({'evt':evt, 'err':'schema'})
            continue
        if is_seen(evt['event_id']):
            logger.info(f
)
            continue
        evt2 = enrich(evt)
        good.append(evt2)
        mark_seen(evt['event_id'])
    return good, bad

ok, ko = process_events(simulated)
len(ok), len(ko)

## 4) Sink: escribir Parquet particionado por fecha

In [ ]:
import pandas as pd
from pathlib import Path

def write_parquet(events):
    if not events:
        return None
    df = pd.DataFrame(events)
    df['date'] = pd.to_datetime(df['ts']).dt.date.astype(str)
    for d, part in df.groupby('date'):
        part_dir = Path(OUT_DIR) / f'date={d}'
        part_dir.mkdir(parents=True, exist_ok=True)
        fp = part_dir / f'events_{int(time.time())}.parquet'
        part.drop(columns=['date']).to_parquet(fp, index=False)
    return True

write_parquet(ok)

## 5) Métricas y logging

In [ ]:
import time
total = len(simulated)
validos = len(ok)
invalidos = len(ko)
metricas = f'total {total}
validos {validos}
invalidos {invalidos}
'
with open(os.path.join(OUT_DIR, 'metrics.txt'), 'w') as f:
    f.write(metricas)
logger.info(f'metrics total={total} validos={validos} invalidos={invalidos}')
metricas

## 6) Consumidor Kafka (opcional en vivo)

In [ ]:
def consume_kafka(max_messages=100):
    try:
        from kafka import KafkaConsumer
        consumer = KafkaConsumer(
            KAFKA_TOPIC,
            bootstrap_servers=KAFKA_BOOTSTRAP,
            auto_offset_reset='earliest',
            enable_auto_commit=False,
            value_deserializer=lambda v: json.loads(v.decode('utf-8'))
        )
        batch = []
        for i, msg in enumerate(consumer):
            batch.append(msg.value)
            if len(batch) >= 50 or i+1 >= max_messages:
                ok, ko = process_events(batch)
                write_parquet(ok)
                # commit offsets al final del batch
                consumer.commit()
                logger.info(f'batch size={len(batch)} ok={len(ok)} ko={len(ko)}')
                batch = []
                if i+1 >= max_messages:
                    break
        consumer.close()
    except Exception as e:
        print('Kafka no disponible, salta esta sección:', e)

# consume_kafka(200)  # Descomentar para una prueba en vivo

## 7) Buenas prácticas y extensiones

- Reintentos con DLQ (cola de mensajes de errores) y trazabilidad por `event_id`.
- Idempotencia con checkpoint durable (SQLite/Redis/DB) y caducidad.
- Backpressure: controlar tamaño de batch y límites de latencia.
- Observabilidad: exportar métricas a Prometheus y logs estructurados.
- Seguridad: evitar PII en logs; cifrado en tránsito y at-rest donde aplique.